In [6]:
import pandas as pd
import pickle
from collections import Counter

import xgboost as xgb
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix



In [7]:

pd.set_option('display.max_rows', 300)

survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

list(survey.columns)

survey.JobSatisfaction.value_counts()



survey.head()

survey.EmploymentStatus.unique()

survey.groupby('EmploymentStatus').count()

# survey.UniversityImportance.value_counts()

# survey.JobSatisfaction.value_counts()




/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,GenderSelect,Country,Age,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
EmploymentStatus,,,,,,,,,,,,,,,,,,,,,
Employed full-time,10897,10853,10840,10702,0,0,10843,2400,9787,9533,...,1066,1060,1061,1062,1062,1068,1065,1063,1065,1075
Employed part-time,917,910,907,901,0,0,903,288,787,751,...,132,131,132,131,132,133,132,134,133,132
I prefer not to say,420,404,399,382,380,383,0,0,0,0,...,190,188,189,187,187,188,188,187,187,187
"Independent contractor, freelancer, or self-employed",1330,1321,1321,1302,0,0,1322,281,1176,1142,...,105,104,104,104,104,106,106,103,104,105
"Not employed, and not looking for work",924,919,915,909,900,901,0,0,0,0,...,522,522,519,521,519,525,520,521,524,520
"Not employed, but looking for work",2110,2098,2096,2075,0,0,0,0,0,0,...,1422,1411,1409,1406,1390,1404,1398,1393,1397,1405
Retired,118,116,117,114,0,0,118,43,80,78,...,0,0,0,0,0,0,0,0,0,0


,Unnamed: 0,GenderSelect,Country,Age,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
EmploymentStatus,,,,,,,,,,,,,,,,,,,,,
Employed full-time,10897,10853,10840,10702,0,0,10843,2400,9787,9533,...,1066,1060,1061,1062,1062,1068,1065,1063,1065,1075
Employed part-time,917,910,907,901,0,0,903,288,787,751,...,132,131,132,131,132,133,132,134,133,132
I prefer not to say,420,404,399,382,380,383,0,0,0,0,...,190,188,189,187,187,188,188,187,187,187
"Independent contractor, freelancer, or self-employed",1330,1321,1321,1302,0,0,1322,281,1176,1142,...,105,104,104,104,104,106,106,103,104,105
"Not employed, and not looking for work",924,919,915,909,900,901,0,0,0,0,...,522,522,519,521,519,525,520,521,524,520
"Not employed, but looking for work",2110,2098,2096,2075,0,0,0,0,0,0,...,1422,1411,1409,1406,1390,1404,1398,1393,1397,1405
Retired,118,116,117,114,0,0,118,43,80,78,...,0,0,0,0,0,0,0,0,0,0


In [8]:
conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
                survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
             ]
choices = [1,0]
survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)


survey = survey[survey.JobSatisfaction.notna()]

In [9]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
# missing_value_df

In [10]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<40]

seventy_five_data

survey_minimal_missing= survey[list(seventy_five_data.columnname)]

survey_minimal_missing.reset_index(inplace=True)
X = survey_minimal_missing.drop(columns = 'JobSatisfaction',axis =1)
y = survey_minimal_missing['JobSatisfaction']
# X

In [11]:
X_cat = X.select_dtypes(include='object')
X_cat = pd.DataFrame(X_cat,dtype ='str')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed = imp_freq.fit_transform(X_cat)
X_cat_imputed = pd.DataFrame(data = X_cat_imputed)

OE = OneHotEncoder()
X_cat_imputed = OE.fit_transform(X_cat_imputed)
X_cat_imputed = pd.DataFrame(data = X_cat_imputed.toarray())


X_num = X.select_dtypes(include='float64')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed = imp_freq.fit_transform(X_num)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)

scale = StandardScaler()
X_num_imputed = scale.fit_transform(X_num_imputed)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)


X_imputed = pd.concat([X_cat_imputed,X_num_imputed],axis =1)

y =pd.Series(y, dtype ='int')

In [12]:
# X_imputed.sample(20)

In [13]:
X, X_test, y, y_test = train_test_split(X_imputed, y, test_size=.2, random_state=100)

In [14]:
from sklearn.svm import SVC

In [5]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
#                        scoring='roc_auc', verbose = True, njobs = -1)

svc = SVC()
svc.fit(X, y)




NameError: name 'X' is not defined

NameError: name 'X' is not defined

In [15]:
confusion_matrix(y_test,clf.predict(X_test))

NameError: name 'clf' is not defined

NameError: name 'clf' is not defined

In [16]:
from sklearn.linear_model import LogisticRegressionCV
logRegModel = LogisticRegressionCV(cv =5, Cs=10,scoring = 'precision')
logRegModel.fit(X,y)

/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:7

/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:7

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0)

In [17]:
confusion_matrix(y_test,logRegModel.predict(X_test))

array([[216, 309],
       [139, 672]])

array([[216, 309],
       [139, 672]])